In [1]:
from pyvis.network import Network
import pandas as pd
import json

In [2]:
 # Opening JSON files
with open('../api_flats/devices/phydot11_accesspoints.json', 'r') as openfile:
    aps = json.load(openfile)

with open('../api_flats/phy_phy80211_ssids_views_ssids.json', 'r') as openfile:
    ssids = json.load(openfile)

with open('../api_flats/devices/phy-IEEE802.11.json', 'r') as openfile:
    wlan = json.load(openfile)

In [3]:
aps_df = pd.DataFrame()

for ap in aps:
    try:
        ssid = ap['dot11.device']['dot11.device.responded_ssid_map'][0]['dot11.advertisedssid.ssid']
    except:
        try:
            ssid = ap['dot11.device']['dot11.device.advertised_ssid_map'][0]['dot11.advertisedssid.ssid']
        except:
            try:
                ssid = ap['dot11.device']['dot11.device.probed_ssid_map'][0]['dot11.probedssid.ssid']
            except:
                ssid = 'eduroam'
    ap_data = {
        "key": [ap['kismet.device.base.key']], 
        "manuf": [ap['kismet.device.base.manuf']], 
        "macaddr": [ap['kismet.device.base.macaddr']],
        "ssid": [ssid]
    }
    
    aps_df = pd.concat([aps_df, pd.DataFrame(ap_data)], ignore_index=True)

print(aps_df[aps_df['ssid']==''])

                               key                     manuf   
1    4202770D00000000_E8D4AF935D2C           Ruckus Wireless  \
5     4202770D00000000_CD2AF935D2C           Ruckus Wireless   
9    4202770D00000000_8CCDAF935D2C           Ruckus Wireless   
14   4202770D00000000_2CB9BAB3AA6C           Ruckus Wireless   
16   4202770D00000000_FCC88D3A1884           Ruckus Wireless   
22   4202770D00000000_6CD12F935D2C           Ruckus Wireless   
23   4202770D00000000_CCCCAF935D2C           Ruckus Wireless   
27   4202770D00000000_BCD4AF935D2C           Ruckus Wireless   
32   4202770D00000000_7CC98D3A1884           Ruckus Wireless   
33   4202770D00000000_ACD3AF935D2C           Ruckus Wireless   
36   4202770D00000000_ECD4AF935D2C           Ruckus Wireless   
38   4202770D00000000_5CCCAF935D2C           Ruckus Wireless   
41   4202770D00000000_4CD5AF935D2C           Ruckus Wireless   
45   4202770D00000000_CCB18D3A1884           Ruckus Wireless   
51   4202770D00000000_6CC58D3A1884      

In [4]:
# Attempt to fill in missing SSIDs
for ssid in ssids:
    set1 = set(aps_df[aps_df['ssid']=='']['key'].values)
    list2 = ssid['dot11.ssidgroup.responding_devices']
    common = list(set1.intersection(list2))
    for val in common:
        ind = aps_df[aps_df['key']==val].index[0]
        # print(ind)
        aps_df.at[ind, 'ssid'] = ssid['dot11.ssidgroup.ssid']

# Remove empty SSIDs
aps_df = aps_df[aps_df['ssid']!='']

In [5]:
wlan_df = pd.DataFrame()

for device in wlan:    
    # try:
    #     ssid = device['dot11.device']['dot11.device.responded_ssid_map'][0]['dot11.advertisedssid.ssid']
    # except:
    #     try:
    #         ssid = device['dot11.device']['dot11.device.advertised_ssid_map'][0]['dot11.advertisedssid.ssid']
    #     except:
    #         try:
    #             ssid = device['dot11.device']['dot11.device.last_probed_ssid_record'][0]['dot11.probedssid.bssid']
    #         except:
    #             try:
    #                 ssid = device['dot11.device']['dot11.device.probed_ssid_map'][0]['dot11.probedssid.ssid']
    #             except:
    #                 pass
    #             # ssid = 'eduroam'
    #             # pass

    wlan_data = {
        "ap": [device['dot11.device']['dot11.device.last_bssid']],
        "key": [device['kismet.device.base.key']], 
        "manuf": [device['kismet.device.base.manuf']], 
        "macaddr": [device['kismet.device.base.macaddr']],
        # "ssid": [ssid],
    }
    
    wlan_data = pd.DataFrame(wlan_data)

    wlan_df = pd.concat([wlan_df, wlan_data], ignore_index=True)
wlan_df['ap_bool'] = False
# wlan_df.rename(columns={'ap': 'ssid'}, inplace=True)

In [6]:
# Attempt to fill in missing APs
for ap in aps:
    set1 = set(wlan_df[wlan_df['ap']=='00:00:00:00:00:00']['macaddr'].values)
    try:
        list2 = ap['dot11.device']['dot11.device.associated_client_map']
    except:
        continue
    common = list(set1.intersection(list2))
    for val in common:
        ind = wlan_df[wlan_df['macaddr']==val].index[0]
        # print(ind)
        wlan_df.at[ind, 'ap'] = ap['kismet.device.base.macaddr']

In [7]:
# prob = pd.DataFrame()

# ssid = ssids[11]
# # probing devices nodes/edges
# temp_prob = pd.DataFrame(ssid['dot11.ssidgroup.probing_devices'])
# temp_prob['ssid'] = ssid['dot11.ssidgroup.ssid']
# temp_prob.rename(columns={0: 'devices'}, inplace=True)
# temp_prob = temp_prob[['ssid', 'devices']]
# # temp_prob['weight'] = 1
# # temp_prob['ap_bool'] = False
# # Updating device keys with mac addresses
# # temp_prob = pd.merge(temp_prob, wlan_df[['ssid', 'macaddr', 'manuf', 'ap_bool']], left_on='devices', right_on='key', how='inner')
# # temp_prob = temp_prob[columns]
# prob = pd.concat([prob, temp_prob], ignore_index=True)

# temp_prob

In [8]:
ssids_df = pd.DataFrame()
ssids_num = pd.DataFrame()
columns = ['ssid', 'macaddr', 'manuf', 'ap_bool']

for ssid in ssids:
    # ssid device numbers
    ssid_data = {
        "ssid": [ssid['dot11.ssidgroup.ssid']], 
        "num_probing": [ssid['dot11.ssidgroup.probing_devices_len']], 
        "num_responding": [ssid['dot11.ssidgroup.responding_devices_len']]
    }
    
    ssids_num = pd.concat([ssids_num, pd.DataFrame(ssid_data)], ignore_index=True)

    temp_res = pd.DataFrame(columns=columns)
    if (len(ssid['dot11.ssidgroup.responding_devices']) != 0):
        # responding devices devices nodes/edges
        temp_res = pd.DataFrame(ssid['dot11.ssidgroup.responding_devices'])
        temp_res['ssid'] = ssid['dot11.ssidgroup.ssid']
        temp_res.rename(columns={0: 'devices'}, inplace=True)
        temp_res = temp_res[['ssid', 'devices']]
        # temp_res['weight'] = 2
        temp_res['ap_bool'] = True
        # Updating device keys with mac addresses
        temp_res = pd.merge(temp_res, aps_df.loc[:, ~aps_df.columns.isin(['ssid'])], left_on='devices', right_on='key', how='inner')
        temp_res = temp_res[columns]
    
    temp_prob = pd.DataFrame(columns=columns)
    if (len(ssid['dot11.ssidgroup.probing_devices']) != 0 and ssid['dot11.ssidgroup.ssid']!='eduroam'):
        # probing devices nodes/edges
        temp_prob = pd.DataFrame(ssid['dot11.ssidgroup.probing_devices'])
        temp_prob['ssid'] = ssid['dot11.ssidgroup.ssid']
        temp_prob.rename(columns={0: 'devices'}, inplace=True)
        temp_prob = temp_prob[['ssid', 'devices']]
        # temp_prob['weight'] = 1
        # temp_prob['ap_bool'] = False
        # Updating device keys with mac addresses
        temp_prob = pd.merge(temp_prob, wlan_df, left_on='devices', right_on='key', how='inner')
        temp_prob = temp_prob[columns]

    # concat all
    ssids_df = pd.concat([ssids_df, temp_res, temp_prob], ignore_index=True)

# ssids_df = pd.concat([ssids_df, wlan_df], ignore_index=True)

# Remove Aruba Networks HP devices
wlan_df = wlan_df[wlan_df['manuf']!='Aruba Networks HP']
# Remove empty devices with empty APs
wlan_df = wlan_df[wlan_df['ap']!='00:00:00:00:00:00']

## Nicki

In [9]:
device_ap_ssid = pd.merge(wlan_df[['key', 'ap', 'manuf', 'macaddr']], aps_df, left_on='ap', right_on='macaddr', how='left')
device_ap_ssid = device_ap_ssid.rename(columns={'key_x': 'key_device', 'manuf_x': 'manuf_device', 'macaddr_x': 'macaddr_device', 
                               'key_y': 'key_ap', 'manuf_y': 'manuf_ap', 'macaddr_y': 'macaddr_ap' })
device_ap_ssid = device_ap_ssid[['macaddr_device', 'key_device', 'manuf_device', 'macaddr_ap', 'key_ap', 'manuf_ap', 'ssid']]
device_ap_ssid

,macaddr_device,key_device,manuf_device,macaddr_ap,key_ap,manuf_ap,ssid
0,2C:5D:93:6F:D5:F7,4202770D00000000_F7D56F935D2C,Ruckus Wireless,2C:5D:93:6F:D5:F7,4202770D00000000_F7D56F935D2C,Ruckus Wireless,Configure.Me-2FD5F0
1,FA:61:6E:8C:69:E7,4202770D00000000_E7698C6E61FA,Unknown,2C:5D:93:6F:D5:F7,4202770D00000000_F7D56F935D2C,Ruckus Wireless,Configure.Me-2FD5F0
2,2C:5D:93:AF:D4:E8,4202770D00000000_E8D4AF935D2C,Ruckus Wireless,NaN,NaN,NaN,NaN
3,2C:5D:93:6F:D4:E8,4202770D00000000_E8D46F935D2C,Ruckus Wireless,2C:5D:93:6F:D4:E8,4202770D00000000_E8D46F935D2C,Ruckus Wireless,Airwave-5G-4-bvv4vx0us8
4,2C:5D:93:2F:D4:E8,4202770D00000000_E8D42F935D2C,Ruckus Wireless,2C:5D:93:2F:D4:E8,4202770D00000000_E8D42F935D2C,Ruckus Wireless,Airwave-4-bvv4vx0us8
...,...,...,...,...,...,...,...
7269,C6:C1:4A:81:76:0B,4202770D00000000_B76814AC1C6,Unknown,84:18:3A:0D:AF:1C,4202770D00000000_1CAF0D3A1884,Ruckus Wireless,Airwave-5G-8-cu7sx0wv2
7270,9E:77:C9:0F:04:38,4202770D00000000_38040FC9779E,Unknown,NaN,NaN,NaN,NaN
7271,32:1C:85:E6:E3:BF,4202770D00000000_BFE3E6851C32,Unknown,6C:5A:B0:93:9F:92,4202770D00000000_929F93B05A6C,TP-Link Corporation Limited,TP-Link_9F92
7272,42:E6:19:15:35:EA,4202770D00000000_EA351519E642,Unknown,84:18:3A:0D:AF:1C,4202770D00000000_1CAF0D3A1884,Ruckus Wireless,Airwave-5G-8-cu7sx0wv2


# graph viz

In [10]:
wlan_df_sample = wlan_df.sample(n=1000)

In [11]:
# Graph Setup
net = Network(
    notebook=True,
    cdn_resources="remote",
    height="750px", 
    width="100%", 
    bgcolor="#222222", 
    font_color="white",
    # select_menu=True,
    # filter_menu=True,
)

# Physics
net.force_atlas_2based()

# net.repulsion(
#     # node_distance=100,
#     # central_gravity=0.1,
#     spring_length=200,
#     # spring_strength=0.05,
#     damping=0.09,
# )
net.show_buttons(filter_='physics')

images = {'Intel Corporation':'https://raw.githubusercontent.com/anrath/data/master/intel_logo.jpg', 
          'Apple':'https://raw.githubusercontent.com/anrath/data/master/apple_gray_logo.png', 
          'Aruba Networks HP':'https://raw.githubusercontent.com/anrath/data/master/aruba_logo.jpg'
        }

In [12]:
# SSID-AP Edges

sources = ssids_df['ssid']
targets = ssids_df['macaddr']
manuf = ssids_df['manuf']
ap_bool = ssids_df['ap_bool']

edge_data = zip(sources, targets, manuf, ap_bool)

for e in edge_data:
    src = e[0]
    dst = e[1]
    manuf = e[2]
    ap_bool = e[3]

    net.add_node(src, src, title=src, group=1, shape='diamond')

    device_group = 2 if ap_bool else 3

    if (manuf in images.keys()):
        net.add_node(dst, dst, title=dst, group=device_group, shape='circularImage', image=images[manuf], brokenImage='circle')
    else:
        net.add_node(dst, dst, title=dst, group=device_group, shape='circularImage', image='https://raw.githubusercontent.com/anrath/data/master/blank.png', brokenImage='circle')
    net.add_edge(src, dst)


In [13]:
# ssids_df[(ssids_df['ssid'].isin(['eduroam', 'UVA WiFi Setup'])) & (ssids_df['manuf']!= 'Aruba Networks HP')]

In [14]:
# Weird apply device?
# ssids_df[ssids_df['macaddr'] =="00:25:00:FF:94:73"]
# wlan_df[wlan_df['ap'] =="00:25:00:FF:94:73"]

In [15]:
# AP-Device Edges
aps = wlan_df_sample['ap']
devices = wlan_df_sample['macaddr']
manuf = wlan_df_sample['manuf']
# ap_bool = wlan_df['ap_bool']

edge_data = zip(aps, devices, manuf)

for e in edge_data:
    src = e[0]
    dst = e[1]
    manuf = e[2]

    # net.add_node(src, src, title=src, group=2, shape='diamond')
    net.add_node(src, src, title=src, group=2, shape='circularImage', image=images['Aruba Networks HP'], brokenImage='circle')

    device_group = 3

    if (manuf in images.keys()):
        net.add_node(dst, dst, title=dst, group=device_group, shape='circularImage', image=images[manuf], brokenImage='circle')
    else:
        net.add_node(dst, dst, title=dst, group=device_group, shape='circularImage', image='https://raw.githubusercontent.com/anrath/data/master/blank.png', brokenImage='circle')
    net.add_edge(src, dst)


In [16]:
# Hover Text
neighbor_map = net.get_adj_list()

# add neighbor data to node hover data
for node in net.nodes:
    node["title"] += " Neighbors:\n" + "\n".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

net.prep_notebook()
net.show("ssid_flats.html")
# net.show("ssid_small.html")

ssid_flats.html
